In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import os
import tqdm
import glob
import tensorflow

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import gray2rgb

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras import optimizers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
import keras.backend as K
from typing import Optional

In [21]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2,

        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1./255
                                  )

In [23]:
train_dataset  = train_datagen.flow_from_directory(directory = '/content/drive/MyDrive/Colab Notebooks/MCI-CN/MCI-CN/train',
                                                   target_size = (224,224),
                                                   class_mode = 'binary',
                                                   subset = 'training',
                                                   batch_size = 32)

Found 858 images belonging to 2 classes.


In [24]:
test_dataset = test_datagen.flow_from_directory(directory = '/content/drive/MyDrive/Colab Notebooks/MCI-CN/MCI-CN/test',
                                                  target_size = (224,224),
                                                  class_mode = 'binary',
                                                  batch_size = 32)

Found 286 images belonging to 2 classes.


In [25]:
base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')


In [26]:
for layer in base_model.layers:
    layer.trainable=False


In [27]:
model=Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1, activation='sigmoid'))

In [28]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [29]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 5) 

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [30]:

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]


In [31]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=METRICS)

In [36]:
history=model.fit(train_dataset,
                        epochs = 30,
                        verbose = 1,
                         callbacks=lr_scheduler)

Epoch 1/30
27/27 [==============================] - 15s 536ms/step - loss: 0.3464 - accuracy: 0.8531 - precision: 0.9257 - recall: 0.8025 - auc: 0.9122 - f1_score: 0.8586 - lr: 0.0100
Epoch 2/30
27/27 [==============================] - 14s 525ms/step - loss: 0.3803 - accuracy: 0.8508 - precision: 0.9274 - recall: 0.7963 - auc: 0.8975 - f1_score: 0.8579 - lr: 0.0063
Epoch 3/30
27/27 [==============================] - 14s 527ms/step - loss: 0.3761 - accuracy: 0.8531 - precision: 0.9298 - recall: 0.7983 - auc: 0.9001 - f1_score: 0.8574 - lr: 0.0040
Epoch 4/30
27/27 [==============================] - 14s 530ms/step - loss: 0.3159 - accuracy: 0.8706 - precision: 0.9322 - recall: 0.8295 - auc: 0.9273 - f1_score: 0.8758 - lr: 0.0025
Epoch 5/30
27/27 [==============================] - 14s 532ms/step - loss: 0.3095 - accuracy: 0.8788 - precision: 0.9394 - recall: 0.8378 - auc: 0.9329 - f1_score: 0.8860 - lr: 0.0016
Epoch 6/30
27/27 [==============================] - 14s 532ms/step - loss: 0.313

In [37]:
scores = model.evaluate(test_dataset)

9/9 [==============================] - 1s 149ms/step - loss: 0.2367 - accuracy: 0.9196 - precision: 0.9808 - recall: 0.8293 - auc: 0.9783 - f1_score: 0.8969


In [38]:
print("Accuracy = ", scores[1])
print("Precision = ", scores[2])
print("Recall = ", scores[3])
print("AUC = ", scores[4])
print("F1_score = ", scores[5])

Accuracy =  0.9195803999900818
Precision =  0.9807692170143127
Recall =  0.8292682766914368
AUC =  0.9783031940460205
F1_score =  0.8969290256500244
